# Assessment Problems

## Imports

In [1]:
# NumPy.
import numpy as np

## Problem 1: Binary Words and Operations

### Technical Glossary
This section defines operations & technical terms used throughout the implementation of solutions for the problems set out in the assignment.

### Bitwise Logical Operations

**Bitwise Operations** are operations that work on the individiual bits of a binary number. Applying the operator to each bit individually but all bits simulatenously.

**XOR (Exclusive OR)**
Outputs 1 when the input bits are different.

| A | B | A ⊕ B|
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   1   |
| 1 | 0 |   1   |
| 1 | 1 |   0   |

**AND**
Outputs 1 when both input bits are 1

| A | B | A & B|
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   0   |
| 1 | 0 |   0   |
| 1 | 1 |   1   |

**NOT**
Output flips the bits

| A | ¬A|
|---|---|
|0  |1  |
|1  |0  |

**Bit Manipulation Operations**
* `ROTR^n(x)`: Rotate right - shifts bits to the right n times, 'fallen' bits move back to the left (start)
* `SHR^n(x)`: Logical shift right - shifts bits to the right n times; fills left with zeros

**Common Terms**
* **Bitwise Operation:** Performs an operation on each individual bit
* **32-bit unsigned integer:** An integer represented by 32 individual bits in a binary sequence. "Unsigned" meaning non-negative values.
* **Diffusion:** Where the input changes are spread throughout the output

See: [FIPS 180-4, Sections 2.2.2 and 3.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

### Implementing Parity
**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Parity}(x, y, z) = x \oplus y \oplus z$

**How it works:**
Chains $'\oplus'$ operations across three 32-bit integers.

Since $'\oplus'$ is a binary operation it takes 2 inputs. The expression is evaluated as $ \text (x \oplus y \oplus z)$
. The result of $x \oplus y$ is then, result $\oplus z$.

Each output bit is 1 if there is an odd number of input bits set to 1, otherwise 0. This is why its called "Parity" it checks the parity(odd/even count) of bits at each position.

**Example**

X = 1100

Y = 1100

Z = 0110

Output = 0110

In [2]:
def parity(x, y, z) -> np.uint32:
    """
    Calculate the parity (bitwise XOR) of three 32-bit unsigned integers.

    Args:
        x, y, z (int): 32-bit unsigned integer.

    Returns:
        int: The parity of the three integers as a 32-bit unsigned integer.

    """
    
    # Loop over each passed parameters
    for args in (x, y, z):
        # Use isInstance to ensure correct type (object, desired type)
        # See: https://www.w3schools.com/python/ref_func_isinstance.asp
        if not isinstance(args, (int, np.integer)):
            # Raises / Throws a TypeError with a message
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast all inputs as NumPy unsigned 32-bit integers.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Apply bitwise XOR (^) to the three inputs and return
    return x ^ y ^ z

In [3]:
def testing_parity():
    assert parity(1, 0, 0) == np.uint32(1)
    assert parity(1, 1, 0) == np.uint32(0)
    assert parity(7, 13, 5) == np.uint32(15)

def test_error_on_parity():
    try:
        parity(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_parity()
test_error_on_parity()

print("All test cases pass on Parity")

All test cases pass on Parity


### Implementing Choose (Ch)
**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x  \land  z)$

__How it works:__
The $\text{Ch}(x, y, z)$ uses x as a selector mask. For each bit position, if x is 1, the output takes the bit from y.
If x is 0, the output takes the bit from z.

This is achieved by calculating $\text (x \land y)$ to select bits from y where x is 1, then $\text (\lnot x \land y)$ to select bits from z where x is 0. Then $'\oplus'$ the results together.

**Example**

X = 1110 

Y = 1010 

Z = 1111 

Output = 1011

In [4]:
def ch(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Choose: return bits from y where x is 1, and bits from z where x is 0.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'masked' value of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over the parameters passed
    for args in (x, y, z):
        # Use isinstance + raise TypeError as before
        # See: Problem 1 -> Implementing Parity
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation and return the value
    return (x & y) ^ (~x & z)

In [5]:
def testing_ch():
    assert ch(1, 0, 0) == np.uint32(0)
    assert ch(1, 3, 6) == np.uint32(7)
    assert ch(3, 5, 8) == np.uint32(9)

def test_error_on_ch():
    try:
        ch(2.5, 4, 9)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_ch()
test_error_on_ch()

print("All test cases pass on Ch")

All test cases pass on Ch


### Implementing Maj
**Purpose:** The Maj computes the bitwise (XOR, AND) of 3 unsigned 32-bit integers. 

**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Maj}(x, y, z) = (x \land y) \oplus (x \land  z) \oplus (y \land  z)$

**How it works:**
The $\text{Maj}(x, y, z)$ implements majority voting at each bit position. For each bit, the output is 1 if at least two input bits are 1, otherwise 0.
This is computed by taking the $'\land'$ of each pair of inputs 
$\text (x \land y)$, $\text (x \land z)$, and $\text (y \land z)$, then $'\oplus'$ the three results together.

**Example:**

X = 11101110

Y = 10100101

Z = 11111001

Output = 11101101

In [6]:
def maj(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Majority: output bit is 1 if two or more input bits are 1.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'voted' values of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over args, use isinstance & raise TypeError as before
    # See: Problem 1 -> Implementing Parity
    for args in (x, y, z):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation & return the value
    return (x & y) ^ (x & z) ^ (y & z)

In [7]:
def testing_maj():
    assert maj(1, 2, 3) == np.uint32(3)
    assert maj(3, 4, 7) == np.uint32(7)
    assert maj(5, 8, 10) == np.uint32(8)

def test_error_on_maj():
    try:
        maj(2, 4.7, 6)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_maj()
test_error_on_maj()

print("All test cases pass for Maj")

All test cases pass for Maj


#### Implementing Big & Small Sigmas with Helper Functions
### Helper Functions

__Purpose:__
Support functions used in both big and small sigmas.

* `ROTR(x, n)`: Rotate right - shifts bits to the right n times, 'fallen' bits move back to the left (start)
* `SHR(x, n)`: Logical shift right - shifts bits to the right n times; fills left with zeros

__Spec.__ Defined in the Secure Hash Standard (FIPS 180-4, p.8-9) as:

$\text {ROTR}^n(x) = (x \gg n) \lor (x \ll (w - n))$

$\text {SHR}^n(x) = x \gg n$

See: [FIPS 180-4, Sections 2.2.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

__Intuition:__
Both functions provide structured bit-mixing.

* Rotate shifts the bits without losing information.
* Logical shift loses information but adds to diffusion.

__Example__

x = 10110011

ROTR(x, 2) = 11101100

SHR(x, 4) = 00001011

In [8]:
def ROTR(x: int, n: int) -> np.uint32:
    """
    Perform a right rotation on a 32-bit unsigned integer.

    Parameters
    ----------
        x : int
            32-bit unsigned integer to be rotated.
        n : int
            Number of positions to rotate.

    Returns
    -------
        numpy.uint32
            The result of the right rotation as a 32-bit unsigned integer.
    
    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Type check as before
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Early return if n is 0
    if n == 0:
        return x
    
    # Ensure n is within the range of 0-31
    n = n % 32

    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the rotation and return the value
    return (x >> n) | (x << (32 - n))

In [9]:
def SHR(x: int, n: int) -> np.uint32:
    """
    Perform a logical right shift on a 32-bit unsigned integer.

    Parameters
    ----------
        x : int 
            32-bit unsigned integer to be shifted.
        n : int
            Number of positions to shift.

    Returns
    -------
        numpy.uint32
            The result of the logical right shift as a 32-bit unsigned integer.
    """

    # Type check as before
    for args in (x, n):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
        
    # Early return if n is 0
    if n == 0:
        return x
    
    # Ensure n is within the range of 0-31
    n = n % 32

    # Cast value to ensure correct type and prevent overflow errors
    x = np.uint32(x)

    # Perform the logical right shift and return the value
    return x >> n

In [10]:
# Same format for tests as previous sub-problems
def test_ROTR_valid():
    assert ROTR(10, 1) == np.uint32(5)
    assert ROTR(15, 2) == np.uint32(3221225475)
    assert ROTR(18, 3) == np.uint32(1073741826)

def test_SHR_valid():
    assert SHR(10, 1) == np.uint32(5)
    assert SHR(13, 2) == np.uint32(3)

def test_ROTR_type_error():
    try:
        ROTR(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

def test_SHR_type_error():
    try:
        SHR(1.5, 2)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")

test_ROTR_valid()
test_SHR_valid()
test_ROTR_type_error()
test_SHR_type_error()

print("All test cases pass on ROTR & SHR")

All test cases pass on ROTR & SHR


#### Big Sigma 0 - $\Sigma_0(x)$

**Purpose**:
Provides non-linear mixing by combining three rotated versions of the input. This ensures the bits are widely spread across. Increasing diffusion.
    
**Spec.** Defined in the Secure Hash Standard as:
    
> $\Sigma_0(x) = \text{ROTR}^{2}(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**Intuition:**

Rotating the words by different amounts makes sure each bit affect multiple output bit positions.
XOR-ing the results ensures the output is very complex to reverse.

**Example**

x = 10110011

Each bit is rotated **2**, **13** & **22** times, then XOR'ed together.

These numbers are **not** arbitrary they are a result of testing during the creation of the Secure Hash Standard.

They reinforce the avalanche effect & ensure good spread diffusion and mixing.

In [11]:
def big_sigma_0(x: int) -> np.uint32:
    """
    Implementing the Big Sigma 0 function used in SHA-256.

    Parameters
    ----------
        x : int
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of ROTR(2) XOR ROTR(13) XOR ROTR(22)

    Raises
    ------
        TypeError
            If the argument is not an integer.
    """
    
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type
    x = np.uint32(x)

    # Perform the Big Sigma 0 operation and return the value
    return np.uint32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

#### Big Sigma 1 - $\Sigma_1(x)$

**Purpose**:
Exact same function as Sigma 0, except with different rotation constants, providing another layer of complexity, diffusion and mixing.
    
**Spec.** Defined in the Secure Hash Standard as:
    
> $\Sigma_1(x) = \text{ROTR}^{6}(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**Intuition:**

Rotating the words by different amounts makes sure each bit affect multiple output bit positions.
XOR-ing the results ensures the output is very complex to reverse.

**Example**

x = 10110011

Each bit is rotated **6**, **11** & **25** times, then XOR'ed together.

These numbers are **not** arbitrary they are a result of testing during the creation of the Secure Hash Standard.

They reinforce the avalanche effect & ensure good spread diffusion and mixing.

In [12]:
def big_sigma_1(x: int) -> np.uint32:
    """
    Implementing the Big Sigma 1 function used in SHA-256.
        
    Parameters
    ----------
        x : int
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of the Big Sigma 1 function as a 32-bit unsigned integer.
    
    Raises
    ------
        TypeError
            If the argument is not an integer.
    """
    
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Big Sigma 1 operation and return the value
    return np.uint32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

In [13]:
def test_big_sigma_0_valid():
    assert big_sigma_0(1)  == np.uint32(ROTR(1, 2) ^ ROTR(1, 13) ^ ROTR(1, 22))
    assert big_sigma_0(10) == np.uint32(ROTR(10, 2) ^ ROTR(10, 13) ^ ROTR(10, 22))

def test_big_sigma_1_valid():
    assert big_sigma_1(7)  == np.uint32(ROTR(7, 6) ^ ROTR(7, 11) ^ ROTR(7, 25))
    assert big_sigma_1(18) == np.uint32(ROTR(18, 6) ^ ROTR(18, 11) ^ ROTR(18, 25))


def test_big_sigma_type_error():
    try:
        big_sigma_0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        big_sigma_1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_big_sigma_0_valid()
test_big_sigma_1_valid()
test_big_sigma_type_error()

print("All test cases pass on Big Sigma 0 & 1")

All test cases pass on Big Sigma 0 & 1


#### Small Sigma 0 - $\sigma_0(x)$
* __Small Sigma 0__  -> $'\oplus'$ Bitwise, 2 rotations (7, 18) & Bitwise shift right (3) - SHA-256 Reference required here

**Spec.** Defined in the Secure Hash Standard as:
> $\sigma_0(x) = \text{ROTR}^{7}(x) \oplus \text{ROTR}^{18}(x) \oplus \text{SHR}^{3}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [14]:
def small_sigma_0(x: int) -> np.uint32:
    """
    Implementing the Small Sigma 0 function used in SHA-256.

    Parameters
    ----------
        x : int
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of the Small Sigma 0 function as a 32-bit unsigned integer.

    Raises
    ------
        TypeError
            If the argument is not an integer.
    """
    
    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")
    
    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Small Sigma 0 operation and return the value
    return np.uint32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))

#### Small Signma 1 - $\sigma_1(x)$
* __Small Sigma 1__  -> $'\oplus'$ Bitwise, 2 rotations (17, 19) & Bitwise shift right (10) - SHA-256 Reference required here

**Spec.** Defined in the Secure Hash Standard as:
> $\sigma_1(x) = \text{ROTR}^{17}(x) \oplus \text{ROTR}^{19}(x) \oplus \text{SHR}^{10}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [15]:
def small_sigma_1(x: int) -> np.uint32:
    """
    Implementing the Small Sigma 1 function used in SHA-256.

    Parameters
    ----------
        x : int 
            32-bit unsigned integer.

    Returns
    -------
        numpy.uint32
            Result of the Small Sigma 1 function as a 32-bit unsigned integer.
    
    Raises
    ------
        TypeError
            If the argument is not an integer.
    """

    # Type check as before
    if not isinstance(x, (int, np.integer)):
        raise TypeError("Argument must be an integer. No other types are allowed.")

    # Cast to np.uint32 to ensure correct type as before
    x = np.uint32(x)

    # Perform the Small Sigma 1 operation and return the value
    return np.uint32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

In [ ]:
def test_small_sigma_0_valid():
    assert small_sigma_0(1)  == np.uint32(ROTR(1, 7) ^ ROTR(1, 18) ^ SHR(1, 3))
    assert small_sigma_0(10) == np.uint32(ROTR(10, 7) ^ ROTR(10, 18) ^ SHR(10, 3))

def test_small_sigma_1_valid():
    assert small_sigma_1(5)  == np.uint32(ROTR(5, 17) ^ ROTR(5, 19) ^ SHR(5, 10))
    assert small_sigma_1(12)  == np.uint32(ROTR(12, 17) ^ ROTR(12, 19) ^ SHR(12, 10))

# Error on this function like using floating numbers
def test_small_sigma_type_error():
    try:
        small_sigma_0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        small_sigma_1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_small_sigma_0_valid()
test_small_sigma_1_valid()
test_small_sigma_type_error()

print("All test cases pass Small Sigma 0 & 1")

All test cases pass Small Sigma 0 & 1


## End of Problem 1

## Problem 2: Fractional Parts of Cube Roots

Use numpy to calculate the constants listed at the bottom of page 11 of the Secure Hash Standard, following the steps below.
These are the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers.

### Instruction
- Write a function called `primes(n)` that generates the first n prime numbers.
- Use the function to calculate the cube root of the first 64 primes.
- For each cube root, extract the first thirty-two bits of the fractional part.
- Display the result in hexadecimal.
- Test the results against what is in the Secure Hash Standard.

### Solution:

In [17]:
# Plots.
import matplotlib.pyplot as plt
# Already import numpy

## Primes Function
**Purpose:** Every integer greater than 1 can be represented uniquely as a product of prime numbers. This could generates the 1st 64 prime numbers. Start with the 1st prime number and all the way to the 64th prime number. 

**Example:** 
* 2, 3, 5 is a prime number.
* 4, 10, 12 is NOT a prime number because it's a factor number. Like 10 / 2 = 5
* 0, 1 is a Non-Prime number

In [18]:
# Brute force test for primality.
def primes(n):
    """Test whether n is prime."""
    # Loop through 2...n-1.
    for i in range(2, n):
        # Calculate remainder of n divided by i.
        if n % i == 0:
            # If this is zero, then n is not prime.
            return False
            
    # If we get here, then n is prime.
    return True

## Cube Root
**Purpose:** After getting the first 64 prime numbers, collect the first 32 bits of fractional part of cube roots.

**Example:**
> $\text{root} = {prime}^{1/3}$

> $\text{frac} = {root} - {int(root)}$

> $\text{frac} = {frac}({2}^{32})$

## End of Problem 2

## Problem 3: Padding

### Instruction

- Write a [generator function](https://realpython.com/introduction-to-python-generators/) `block_parse(msg)` that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard.
-The function should accept a [bytes object](https://realpython.com/python-bytes/) called `msg`.
- At each iteration, it should yield the next 512-bit block of `msg` as a bytes object.
- Ensure that the final block (or final two blocks) include the required padding of `msg` as specified in the standard.
- Test the generator with messages of different lengths to confirm proper padding and block output.

### Solution:

## End of Problem 3

## Problem 4: Hashes

### Instruction
- Write a function `hash(current, block)` that calculates the next hash value given the `current` hash value and the next message `block` according to section *6.2.2 SHA-256 Hash Computation* on page 22 of the Secure Hash Standard.

### Solution:

## End of Problem 4

## Problem 5: Passwords

### Instruction

- The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm.
- As strings, they were encoded using [UTF-8](https://en.wikipedia.org/wiki/UTF-8).
- Determine the passwords and explain how you found them.
- Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

### SHA-256 Hashes

1. `5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8`
2. `873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34`
3. `b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342`

### Solution:

## End of Problem 5

# End of the Assessment